## what to do next

class bitmexDataHandler
- bm_ws_market  订阅 成交明细，买一卖一
- snapshot()   500毫秒一个切片，发送MarketEvent

其中bm_ws_market中的如下成员有用：
- prev_bar
- current_bar
- bid1, ask1

todo:
1. utils.calc_ts()
2. 发布订阅 bar_type, bm_ws_market.subscribe('XBTUSD', ['1m', '5m'], false)
3. bar和last_price需要有两个时间戳：交易所时间和本机实际收到的时间
4. 定义一些全局变量，eg. 枚举类 BarType, def n_minutes()获取对应的分钟数

bar_type 可以拼成任何时间周期，eg. 9m, 17m, 18m, 可以有shift   
所以bar_type的格式是这样的： '9m3s', '18m25s'   
trading_day, ts = 20180101, 190015   
目前可以实现的： trading_day, ts = calc_td_ts()， ts保留至秒钟

In [2]:
# MarketEvent需定义格式：

class MarketEvent(object):
    def __init__(self, channel, market_event_type):
        self.etype ='MARKET'
        self.channel = channel
        self.market_event_type = market_event_type

# channel:
# symbol:bar_type
# eg. XBTUSD:1m, XBTUSD:snapshot, XBTUSD:tick

# market_event_type:
# BAR_OPEN, BAR_CLOSE, 
# SNAPSHOT, （格式同国内500ms切片，open,hightest,lowest,last,volume,oi,bid1,ask1）
# TICK,
# OB_TOP, OB_10, OB_ALL

# 注意 MARKET_EVENT 只是提示“有事件”，但它自身不带data

In [180]:
a = () 
isinstance(a, tuple)

True

## GlobalSettings.py

In [197]:
# qsSettings.py
import json


class GlobalSettings(object):
    """全局设置"""
    
    def  __init__(self):
        self.account = None
        self.apiKey = None
        self.apiSecret = None
        self.is_test = True   # always True until real trading
        self.loglevel = 'debug'
        self.logfile = 'default-log.log'
        self.symbols = ['XBTUSD']    # todo: from strategies config
    
    def from_config_file(self, file):
        """读取配置文件"""
        
        with open(file) as f:
            st = json.load(f)
            
        self.account = st['account']['userName']
        self.apiKey = st['account']['apiKey']
        self.apiSecret = st['account']['apiSecret']
        self.loglevel = st['log']['logLevel']
        self.logfile = st['log']['logFile']

## DataHandler.py

In [248]:
class Orderbook(object): 
    def __init__(self, symbol=None, bid1=None, bid1vol=None, ask1=None, ask1vol=None, timestamp=None):
        self.symbol = symbol
        self.bid1 = bid1
        self.bid1vol = bid1vol
        self.ask1 = ask1
        self.ask1vol = ask1vol
        self.timestamp = timestamp
        
    def __repr__(self):
        return self.__dict__.__repr__()
        

class Tick(object): 
    def __init__(self, symbol=None, price=None, volume=None, direction=None, timestamp=None):
        self.symbol = symbol
        self.price = price
        self.volume = volume
        self.direction = direction
        self.timestamp = timestamp
        
    def __repr__(self):
        return self.__dict__.__repr__()
    
class Bar(object):
    pass

class Snapshot(object):
    pass

In [185]:
from bitmexWSMarket import bitmexWSMarket
from bitmexREST import bitmexREST
from utils import generate_logger, calculate_td_ts
import queue


class bitmexDataHandler(object):
    def __init__(self, g, event_q):
        self.g = g                            # global settings
        self.symbols = g.symbols              # 订阅的标的
        self.market_data_q = queue.Queue()    # MarketData 队列（带数据）
        self.event_q = event_q                # 事件队列
        seff.logger.generate_logger('DataHandler', g.loglevel, g.logfile)
        
    def run(self):
        self.__construct_bm_ws_market()
        
        while True:
            try:
                data = self.market_data_q.get(timeout=100)
            except queue.Empty:
                self.logger.warning('no data in market_data_q for 100 seconds')
            else:
                if isinstance(data, Tick):
                    self.processTick(data)
                elif isinstance(data, Orderbook):
                    self.processOrderbook(data)
                else:
                    self.logger.warning('Invalid data type from market_data_q: %s' % data)
                
        
    def __construct_bm_ws_market(self):
        self.bm_ws_market = bitmexWSMarket(apiKey=None, apiSecret=None, 
                                           is_test=self.g.is_test, loglevel=self.g.loglevel, logfile=self.g.logfile)
        self.bm_ws_market.connect()
        self.bm_ws_market.add_market_data_q(self.market_data_q)
        for s in self.symbols:
            self.bm_ws_market.subscribe(s, trade=True, orderbook=True)  # todo subscirbe(symbol, trades, orderbook)
        self.bm_ws_market.wait_for_data()   # todo wait_for_data()
        

    def processTick(self, tick):
        self.logger.debug('Processing Tick... %s' % tick)
        # 1. 更新last_price
        self.__update_last_price(tick)
        
        # 2. if 该symbol订阅了tick事件，推送（全局事件队列）
        if False:
            self.__push_tick_event(tick.symbol)
        
        # 3. 生成bar
        self.__bar(tick)
    
    def processOrderbook(self, ob):
        # 更新Orderbook
        pass
    
    def register_bar_event(self, symbol, bar_type):
        pass
    
    def snapshot(self, symbol):
        """参照国内期货快照数据结构"""
        pass
    
    

## main.py

In [194]:
# 主程序
import queue

In [195]:
# 加载全局设置
g = GlobalSettings()
g.from_config_file('global_settings.json')
g.__dict__

{'account': 'jiaru2015@gmail.com',
 'apiKey': 'ob_a0KcC21NpHnFx7lym9-Jp',
 'apiSecret': 'XHbcrDUt6Aw_sNYLFDto6U4PeurF-uqDK8TE8e15Stgee0gR',
 'is_test': True,
 'logfile': 'test-bitmex-trading.log',
 'loglevel': 'debug',
 'symbols': ['XBTUSD']}

In [196]:
# 事件队列
event_q = queue.Queue()

In [175]:
# DataHandler
datahandler = bitmexDataHandler(g, event_q)
datahandler.register_bar_event('XBTUSD', '1m')
# for s,bar_type in params.subscription:
#     datahandler.register_bar_event(s, bar_type)
datahandler.run()

## bitmexWSMarket2.py

In [256]:
from bitmexWS import bitmexWS
from bitmexREST import bitmexREST
from utils import generate_logger
import time
#from DataHandler import Tick, Orderbook


class bitmexWSMarket2(bitmexWS):
    """bitmexWS subscribing market data (single symbol)"""
    
    def __init__(self, *args, **kwargs):
        """
        self.symbols:
        {
          'XBTUSD': Snapshot(last_price, bid1, ask1, bid1vol, ask1vol, 
                              last_price_ts, orderbook_ts,
                              last_price_receiveTime, orderbook_receiveTime)
        }
        """
        super().__init__(*args, **kwargs)
        self.logger = generate_logger('bitmexWS_Market2')
        self.symbols = {}   
    
    def add_market_data_q(self, q):
        self.market_data_q = q

    def subscribe(self, symbol, trade=True, orderbook=False):
        self.symbols[symbol] = dict(trade=trade, orderbook=orderbook)  # 每订阅一个symbol, 都将其添加进self.symbols字典中
        if trade:
            self.subscribe_topic('trade:%s' % symbol)  # 订阅成交明细
        if orderbook:
            self.subscribe_topic('quote:%s' % symbol)  # 订阅一档委托单簿
        
    def wait_for_data(self, symbols=None, trade=None, orderbook=None):
        """等待第一个数据的到来
        symbols: by default wait for all self.symbols
        trade: if true: 检查last_price是否为None
        orderbook:  if true: 检查bid1&&bid2是否都不为None
        """
        if symbols is None:
            symbols = self.symbols
        if trade is None:
            trade = False
        if orderbook is None:
            orderbook = False
        while True:
            b_trade = not trade or all(self.symbols[s].last_price is not None for s in symbols)
            b_orderbook = not orderbook or all(self.symbols[s].bid1 is not None and self.symbols[s].ask1 is not None for s in symbols)
            if b_trade and b_orderbook:
                return
            self.logger.debug('waiting for data ...')
            time.sleep(1)
        
    def onData(self, msg):   # 不处理partial了
        msg_handler_dict = {
            'quote': self.__process_quote_msg,
            'trade': self.__process_trade_msg,
        }
        tb = msg.get('table')
        if tb in msg_handler_dict:
            func = msg_handler_dict[tb]
            func(msg)
    
    def __process_quote_msg(self, msg):
        """处理quote订阅：
        组装 Orderbook()
        丢进 market_data_q
        """
        quote_s = msg['data']
        for quote in quote_s:
            ob = Orderbook(symbol=quote['symbol'], 
                           bid1=quote['bidPrice'], bid1vol=quote['bidSize'], 
                           ask1=quote['askPrice'], ask1vol=quote['askSize'], 
                           timestamp = quote['timestamp'])
            self.market_data_q.put(ob)
    
    def __process_trade_msg(self, msg):
        """处理trade订阅：
        组装 Tick()
        丢进 market_data_q
        """
        tick_s = msg['data']
        for tick in tick_s:
            tk = Tick(symbol=tick['symbol'],
                      price=tick['price'],
                      volume=tick['size'],
                      direction=tick['side'],
                      timestamp = tick['timestamp'])
            self.market_data_q.put(tk)
    
 
        
if __name__ == '__main__':
    import queue
    import time
    
    market_data_q = queue.Queue()
    
    bm_ws_market = bitmexWSMarket2(apiKey=None, apiSecret=None, 
                                   is_test=g.is_test, loglevel=g.loglevel, logfile=g.logfile)
    bm_ws_market.connect()
    bm_ws_market.add_market_data_q(market_data_q)
    for s in g.symbols:
        bm_ws_market.subscribe(s, trade=True, orderbook=True)
    bm_ws_market.wait_for_data()

    
    print('===============================================')
    while True:
        try:
            a = market_data_q.get(timeout=10)
        except queue.Empty:
            print('waring: no data in 10 sec')
        else:
            print(a)

[2018-10-06 18:08:20 bitmexWS_Market2 INFO] ws thread start
[2018-10-06 18:08:20 bitmexWS_Market2 INFO] ws thread start
[2018-10-06 18:08:20 bitmexWS_Market2 INFO] ws thread start
[2018-10-06 18:08:20 bitmexWS_Market2 INFO] ws thread start
[2018-10-06 18:08:20 bitmexWS_Market2 INFO] ws thread start
[2018-10-06 18:08:22 bitmexWS_Market2 DEBUG] Calling ws.__on_open()
[2018-10-06 18:08:22 bitmexWS_Market2 DEBUG] Calling ws.__on_open()
[2018-10-06 18:08:22 bitmexWS_Market2 DEBUG] Calling ws.__on_open()
[2018-10-06 18:08:22 bitmexWS_Market2 DEBUG] Calling ws.__on_open()
[2018-10-06 18:08:22 bitmexWS_Market2 DEBUG] Calling ws.__on_open()
[2018-10-06 18:08:22 bitmexWS_Market2 INFO] Successful connected to BitMEX WebSocket API
[2018-10-06 18:08:22 bitmexWS_Market2 INFO] Successful connected to BitMEX WebSocket API
[2018-10-06 18:08:22 bitmexWS_Market2 INFO] Successful connected to BitMEX WebSocket API
[2018-10-06 18:08:22 bitmexWS_Market2 INFO] Successful connected to BitMEX WebSocket API
[201

[2018-10-06 18:08:22 bitmexWS_Market2 INFO] Subscribe to trade:XBTUSD
[2018-10-06 18:08:22 bitmexWS_Market2 INFO] Subscribe to trade:XBTUSD
[2018-10-06 18:08:22 bitmexWS_Market2 INFO] Subscribe to trade:XBTUSD
[2018-10-06 18:08:22 bitmexWS_Market2 INFO] Subscribe to trade:XBTUSD
[2018-10-06 18:08:22 bitmexWS_Market2 INFO] Subscribe to trade:XBTUSD
[2018-10-06 18:08:22 bitmexWS_Market2 INFO] Subscribe to quote:XBTUSD
[2018-10-06 18:08:22 bitmexWS_Market2 INFO] Subscribe to quote:XBTUSD
[2018-10-06 18:08:22 bitmexWS_Market2 INFO] Subscribe to quote:XBTUSD
[2018-10-06 18:08:22 bitmexWS_Market2 INFO] Subscribe to quote:XBTUSD
[2018-10-06 18:08:22 bitmexWS_Market2 INFO] Subscribe to quote:XBTUSD


{'direction': 'Sell', 'volume': 100, 'timestamp': '2018-10-06T10:08:19.774Z', 'symbol': 'XBTUSD', 'price': 6564}
{'timestamp': '2018-10-06T10:08:21.534Z', 'ask1vol': 343003, 'bid1': 6564, 'bid1vol': 13235, 'symbol': 'XBTUSD', 'ask1': 6564.5}
{'direction': 'Sell', 'volume': 20, 'timestamp': '2018-10-06T10:08:26.196Z', 'symbol': 'XBTUSD', 'price': 6564}


[2018-10-06 18:08:27 bitmexWS_Market2 DEBUG] >>> send ping
[2018-10-06 18:08:27 bitmexWS_Market2 DEBUG] >>> send ping
[2018-10-06 18:08:27 bitmexWS_Market2 DEBUG] >>> send ping
[2018-10-06 18:08:27 bitmexWS_Market2 DEBUG] >>> send ping
[2018-10-06 18:08:27 bitmexWS_Market2 DEBUG] >>> send ping


{'timestamp': '2018-10-06T10:08:26.196Z', 'ask1vol': 343003, 'bid1': 6564, 'bid1vol': 13215, 'symbol': 'XBTUSD', 'ask1': 6564.5}
{'timestamp': '2018-10-06T10:08:26.380Z', 'ask1vol': 343003, 'bid1': 6564, 'bid1vol': 13280, 'symbol': 'XBTUSD', 'ask1': 6564.5}


KeyboardInterrupt: 

In [257]:
bm_ws_market.exit()

[2018-10-06 18:08:33 bitmexWS_Market2 INFO] Exiting ...
[2018-10-06 18:08:33 bitmexWS_Market2 INFO] Exiting ...
[2018-10-06 18:08:33 bitmexWS_Market2 INFO] Exiting ...
[2018-10-06 18:08:33 bitmexWS_Market2 INFO] Exiting ...
[2018-10-06 18:08:33 bitmexWS_Market2 INFO] Exiting ...
[2018-10-06 18:08:37 bitmexWS_Market2 INFO] ping thread end.
[2018-10-06 18:08:37 bitmexWS_Market2 INFO] ping thread end.
[2018-10-06 18:08:37 bitmexWS_Market2 INFO] ping thread end.
[2018-10-06 18:08:37 bitmexWS_Market2 INFO] ping thread end.
[2018-10-06 18:08:37 bitmexWS_Market2 INFO] ping thread end.
[2018-10-06 18:08:37 bitmexWS_Market2 DEBUG] Calling ws.__on_close()
[2018-10-06 18:08:37 bitmexWS_Market2 DEBUG] Calling ws.__on_close()
[2018-10-06 18:08:37 bitmexWS_Market2 INFO] Exit bitmexWS (intended)
[2018-10-06 18:08:37 bitmexWS_Market2 DEBUG] Calling ws.__on_close()
[2018-10-06 18:08:37 bitmexWS_Market2 INFO] Exit bitmexWS (intended)
[2018-10-06 18:08:37 bitmexWS_Market2 DEBUG] Calling ws.__on_close()
[